### Feature Store Creation

In [3]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col


connection_parameters = {
    "account": "<your snowflake account identifier>",
    "user": "<your snowflake username>",
    "password": "<your snowflake password>",
    "role": "<your snowflake role>",
    "warehouse": "<your snowflake warehouse>",  # optional
    "database": "<your snowflake database>",  # optional
    "schema": "<your snowflake schema>" # optional
  }

# it is best practice to not use/save your credentials in a jupyter notebook
# In this case I'm loading my credentials as a dictionary
import os
connection_parameters = {
    "account": os.getenv('SNOWFLAKE_ACCOUNT'),
    "user": os.getenv('SNOWFLAKE_USER'),
    "password": os.getenv('SNOWFLAKE_PASSWORD'),
    "role": os.getenv('SNOWFLAKE_ROLE'),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),  # optional
    "database": "SNOWPARK",  # optional
    "schema": "TUTORIAL" # optional
  }


session = Session.builder.configs(connection_parameters).create()

In [4]:
#CREATE FEATURE STORE

from snowflake.ml.feature_store import (FeatureStore, FeatureView, Entity, CreationMode)

fs = FeatureStore(
    session=session,
    database="SNOWPARK",
    name="TUTORIAL",
    default_warehouse="COMPUTE_WH",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

### Creating & Registering Feature Entity

In [5]:
entity = Entity(name="WEATHER", join_keys=["ID"])
fs.register_entity(entity)
fs.list_entities().show()

FeatureStore.register_entity() is in private preview since 1.0.8. Do not use it in production. 
FeatureStore.get_entity() is in private preview since 1.0.8. Do not use it in production. 
FeatureStore.list_entities() is in private preview since 1.0.8. Do not use it in production. 


-------------------------------------------------
|"NAME"   |"JOIN_KEYS"  |"DESC"  |"OWNER"       |
-------------------------------------------------
|WEATHER  |["ID"]       |        |ACCOUNTADMIN  |
-------------------------------------------------



In [6]:
df = session.table("model_data")
df.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATETIME"           |"SEASON"  |"HOLIDAY"  |"WORKINGDAY"  |"WEATHER"  |"TEMP"  |"ATEMP"  |"HUMIDITY"  |"WINDSPEED"  |"CASUAL"  |"REGISTERED"  |"COUNT"  |"HOUR"  |"MONTH"  |"DATE"      |"WEEKDAY"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2011-01-01 00:00:00  |1         |0          |0             |1          |9.84    |14.395   |81          |0.0000       |3         |13            |16       |0       |1        |2011-01-01  |6          |
|2011-01-01 01:00:00  |1         |0          |0             |1          |9.02    |13.635   |80          |0.0000       |8         |32            |40       |1       |1        |2011-01-01  |6          |


### Creating & Registering Feature Views

In [7]:
from snowflake.snowpark.types import IntegerType

# CREATING ID COLUMN
from snowflake.snowpark.functions import monotonically_increasing_id
df = df.withColumn("ID", monotonically_increasing_id())

#CREATING A NEW FEATURE
df = df.withColumn("QUARTER", ((df["MONTH"] + 2) / 3).cast(IntegerType()))
df.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DATETIME"           |"SEASON"  |"HOLIDAY"  |"WORKINGDAY"  |"WEATHER"  |"TEMP"  |"ATEMP"  |"HUMIDITY"  |"WINDSPEED"  |"CASUAL"  |"REGISTERED"  |"COUNT"  |"HOUR"  |"MONTH"  |"DATE"      |"WEEKDAY"  |"ID"  |"QUARTER"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2011-01-01 00:00:00  |1         |0          |0             |1          |9.84    |14.395   |81          |0.0000       |3         |13            |16       |0       |1        |2011-01-01  |6          |0     |1          |
|2011-01-01 01:00:00  |1         |0          |0             |1          |9.02    |13.635   |80          |0.0000       |8    

In [8]:
# SELECTING COLUMNS FOR FIRST FEATURE VIEW
feature_df = df.select(["SEASON", "WEATHER", "TEMP", "ATEMP", "HUMIDITY", "WINDSPEED", "QUARTER", "ID"])

feature_df.show()

-----------------------------------------------------------------------------------------
|"SEASON"  |"WEATHER"  |"TEMP"  |"ATEMP"  |"HUMIDITY"  |"WINDSPEED"  |"QUARTER"  |"ID"  |
-----------------------------------------------------------------------------------------
|1         |1          |9.84    |14.395   |81          |0.0000       |1          |0     |
|1         |1          |9.02    |13.635   |80          |0.0000       |1          |1     |
|1         |1          |9.02    |13.635   |80          |0.0000       |1          |2     |
|1         |1          |9.84    |14.395   |75          |0.0000       |1          |3     |
|1         |1          |9.84    |14.395   |75          |0.0000       |1          |4     |
|1         |2          |9.84    |12.880   |75          |6.0032       |1          |5     |
|1         |1          |9.02    |13.635   |80          |0.0000       |1          |6     |
|1         |1          |8.20    |12.880   |86          |0.0000       |1          |7     |
|1        

In [9]:
#CREATING FEATURE VIEW

fv = FeatureView(
    name="WEATHER_FEATURES",
    entities=[entity],
    feature_df=feature_df,
    desc="weather features"
)

fv = fs.register_feature_view(
    feature_view=fv,
    version="V1",
    #refresh_freq="1 minute",
    block=True
)

FeatureStore.register_feature_view() is in private preview since 1.0.8. Do not use it in production. 
FeatureStore.get_feature_view() is in private preview since 1.0.8. Do not use it in production. 


In [10]:
# EXAMINE FEATURE VIEW CONTENT

fs.read_feature_view(fv).show()

FeatureStore.read_feature_view() is in private preview since 1.0.8. Do not use it in production. 


-----------------------------------------------------------------------------------------
|"SEASON"  |"WEATHER"  |"TEMP"  |"ATEMP"  |"HUMIDITY"  |"WINDSPEED"  |"QUARTER"  |"ID"  |
-----------------------------------------------------------------------------------------
|1         |1          |9.84    |14.395   |81          |0.0000       |1          |0     |
|1         |1          |9.02    |13.635   |80          |0.0000       |1          |1     |
|1         |1          |9.02    |13.635   |80          |0.0000       |1          |2     |
|1         |1          |9.84    |14.395   |75          |0.0000       |1          |3     |
|1         |1          |9.84    |14.395   |75          |0.0000       |1          |4     |
|1         |2          |9.84    |12.880   |75          |6.0032       |1          |5     |
|1         |1          |9.02    |13.635   |80          |0.0000       |1          |6     |
|1         |1          |8.20    |12.880   |86          |0.0000       |1          |7     |
|1        

In [11]:
#CREATE RENTAL FEATURES

rental_df = df.select(["HOLIDAY", "WORKINGDAY", "CASUAL", "REGISTERED", "COUNT", "HOUR", "MONTH", "WEEKDAY", "ID"])

rental_fv = FeatureView(
    name="RENTAL_FEATURES",
    entities=[entity],
    feature_df=rental_df,
    desc="rental features"
)
rental_fv = fs.register_feature_view(
    feature_view=rental_fv,
    version="V1",
    #refresh_freq="1 minute",
    block=True
)

In [12]:
# LIST ALL FEATURE VIEW
fs.list_feature_views(entity_name="WEATHER").select(["NAME", "ENTITIES", "FEATURE_DESC"]).show()

FeatureStore.list_feature_views() is in private preview since 1.0.8. Do not use it in production. 


-------------------------------------------------------------------
|"NAME"            |"ENTITIES"              |"FEATURE_DESC"       |
-------------------------------------------------------------------
|RENTAL_FEATURES   |[                       |{                    |
|                  |  {                     |  "CASUAL": "",      |
|                  |    "desc": "",         |  "COUNT": "",       |
|                  |    "join_keys": [      |  "HOLIDAY": "",     |
|                  |      "ID"              |  "HOUR": "",        |
|                  |    ],                  |  "MONTH": "",       |
|                  |    "name": "WEATHER",  |  "REGISTERED": "",  |
|                  |    "owner": null       |  "WEEKDAY": "",     |
|                  |  }                     |  "WORKINGDAY": ""   |
|                  |]                       |}                    |
|WEATHER_FEATURES  |[                       |{                    |
|                  |  {                     |  "

In [13]:
# COMBINING TWO FEATURES

full_fv = fs.merge_features(features=[fv, rental_fv], name="FULL_FEATURES")
full_fv = fs.register_feature_view(feature_view=full_fv, version="V1")

AttributeError: 'FeatureStore' object has no attribute 'merge_features'

### Data Generation through Feature views

In [21]:
#GENERATING TRAINING DATA
spine_df = session.table("MODEL_DATA")
spine_df = spine_df.withColumn("ID", monotonically_increasing_id())
spine_df = spine_df.select("ID", "COUNT")
spine_df.show()


training_data = fs.generate_dataset(
    spine_df=spine_df,
    features=[
        fv.slice([
            "HUMIDITY","QUARTER","SEASON","TEMP","WEATHER","WINDSPEED"
        ])
    ],
    materialized_table="TRAINING_DATA",
    spine_timestamp_col=None,
    spine_label_cols=["COUNT"],
    save_mode="merge",
    exclude_columns=['ID']
)

training_data.df.show()

FeatureStore.generate_dataset() is in private preview since 1.0.8. Do not use it in production. 


------------------
|"ID"  |"COUNT"  |
------------------
|0     |16       |
|1     |40       |
|2     |32       |
|3     |13       |
|4     |1        |
|5     |1        |
|6     |2        |
|7     |3        |
|8     |8        |
|9     |14       |
------------------



SnowparkSQLException: (1300) (1304): 01b393d9-0002-4bba-0000-005ba607a44d: 002003 (42S02): SQL compilation error:
Object 'SNOWPARK.TUTORIAL.WEATHER_FEATURES$V1' does not exist or not authorized.

### Model Training & Prediction Using Enriched Data

In [22]:
# TRAIN A MODEL

from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.modeling.ensemble import GradientBoostingRegressor

FEATURE_LIST = [ "HOLIDAY", "WORKINGDAY", "WEEKDAY", "HOUR", "HUMIDITY", "MONTH", "TEMP", "YEAR", "ATEMP", "WINDSPEED", "SEASON", "WEATHER"]
LABEL_COLUMNS = ['COUNT']
OUTPUT_COLUMNS = ['PREDICTED_COUNT']

param_grid = {
        "n_estimators":[100, 200, 300, 400, 500],
        "learning_rate":[0.1, 0.2, 0.3, 0.4, 0.5],
}

grid_search = GridSearchCV(
    estimator=GradientBoostingRegressor(),
    param_grid=param_grid,
    n_jobs = -1,
    scoring="neg_root_mean_squared_error",
    input_cols=FEATURE_LIST,
    label_cols=LABEL_COLUMNS,
    output_cols=OUTPUT_COLUMNS
)

train_df = training_data.df.drop(["ID"])

NameError: name 'training_data' is not defined

In [ ]:
grid_search.fit(train_df)

In [ ]:
#PREDICT FROM FEATURE STORE GENERATED DATA
test_df = spine_df.limit(3).select("ID")
enriched_df = fs.retrieve_feature_values(
    test_df, training_data.load_features())
enriched_df = enriched_df.drop('ID')
enriched_df.show()

In [ ]:
pred = grid_search.predict(enriched_df)
pred.show()

### Feature Store - Support Functions

In [ ]:
# FEATURE VIEW  FROM LIBRARY

#SLICE - GET SUBSET OF FEATURES FROM A VIEW
sliced_fv = full_fv.slice(["TEMP"])
sliced_fv

In [ ]:
#FEATURE VIEW PHYSICAL NAME
print(full_fv.physical_name())

#FEATURE VIEW QUALIFIED NAME
print(full_fv.fully_qualified_name())

In [ ]:
#SET DESCRIPTION TO FEATURE

full_fv = full_fv.attach_feature_desc({"TEMP": "Current Temperature"})

In [ ]:
# FEATURE STORE FROM LIBRARY

#GET FEATURE VIEW
fs.get_feature_view("FULL_FEATURES", "V1").to_df(session).show()

In [ ]:
#SUSPEND A FEATURE VIEW
fs.suspend_feature_view(full_fv)

#RESUME A FEATURE VIEW
fs.resume_feature_view(full_fv)

fs.get_feature_view("FULL_FEATURES", "V1").to_df(session)["NAME", "STATUS"].show()

In [ ]:
# DELETE A FEATURE VIEW

fs.list_feature_views(entity_name="WEATHER").select(["NAME"]).show()
fs.delete_feature_view(fv)

In [ ]:
#LIST ENTITIES

entity = Entity(name="TEST", join_keys=["ID"])
fs.register_entity(entity)
fs.list_entities().show()

In [ ]:
# GET ENTITY
fs.get_entity("TEST")

In [ ]:
#DELETE ENTITY
fs.delete_entity("TEST")
fs.list_entities().show()

In [ ]:
#load data into feature view from dataset
fs.load_feature_views_from_dataset(training_data)

In [14]:
#CLEAR ALL THE ITEMS IN THE FEATURE STORE
fs.clear()

FeatureStore.clear() is in private preview since 1.0.8. Do not use it in production. 
